In [1]:
# !pip install umap-learn
# !pip uninstall umap -y

In [2]:
# End-End training and testing of the new pytorch-based VGG16-deeplabv3 architecture

In [1]:
# https://stackoverflow.com/questions/37893755/tensorflow-set-cuda-visible-devices-within-jupyter
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# https://stackoverflow.com/questions/56008683/could-not-create-cudnn-handle-cudnn-status-internal-error
import tensorflow as tf
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.95)
config = tf.compat.v1.ConfigProto(gpu_options=gpu_options)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [2]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [3]:
import importlib

import wasserstein_utils
import data_utils
import losses
import networks
import deeplabv3 as dlv3
import utils

In [4]:
import time
import numpy as np

import data_utils
import data_utils_new
import losses

from IPython import display

In [8]:
import umap

/global/homes/f/fangjun/.conda/envs/myenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
backbone = 'vgg16'
dataset = "SYNTHIA_VIPER_COMBINED_13_0331"

# H x W x C
img_shape = (512,1024,3)

# 13 classes + void
num_classes = 14

batch_size=4

do_training = False

epochs=100000
epoch_step=250

num_projections=100

source_data_dir = './processed-data/1024x512/13_classes/SYNTHIA/train/'
cityscapes_data_dir = './processed-data/1024x512/13_classes/VIPER_COMBINED/train/'

fn_w_dlv3 = "weights/" + dataset + "/" + backbone + "_deeplabv3.h5"
fn_w_cls = "weights/" + dataset + "/" + backbone + "_deeplabv3_classifier.h5"

fn_w_adapted_dlv3 = "weights/" + dataset + "/" + backbone +"_deeplabv3_adapted.h5"
fn_w_adapted_cls = "weights/" + dataset + "/" + backbone + "_deeplabv3_classifier_adapted.h5"

In [6]:
importlib.reload(losses)

deeplabv3 = dlv3.deeplabv3(activation=None, \
                           backbone=backbone, \
                           num_classes=num_classes)

X = deeplabv3.input
Y = tf.keras.layers.Input(shape=(img_shape[0], img_shape[1], num_classes,), dtype='float32', name='label_input')

C_in = tf.keras.layers.Input(shape=deeplabv3.layers[-1].output_shape[1:], dtype='float32', name='classifier_input')
classifier = tf.keras.Model(C_in, networks.classifier_layers(C_in, num_classes = num_classes, activation='softmax'))

# A combined model, giving the output of classifier(deeplabv3(X))
combined = tf.keras.Model(X, classifier(deeplabv3(X)))
combined.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False))

# A model outputting hxwx1 labels for each image. Also useful to verify the
# mIoU with Keras' built-in function. Will however also consider the 'ignore' class. 
combined_ = tf.keras.Model(X, tf.cast(tf.keras.backend.argmax(combined(X), axis=-1), 'float32'))
combined_.compile(metrics=[tf.keras.metrics.MeanIoU(num_classes=num_classes)], loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False))

# Set up training
opt = tf.keras.optimizers.Adam(lr=1e-4, epsilon=1e-8, decay=1e-6)
# opt = tf.keras.optimizers.SGD(lr=1e-4, momentum=.9)

loss_function = losses.weighted_ce_loss(num_classes, 0)
wce_loss = loss_function(Y, classifier(deeplabv3(X)), from_logits=False)

# https://stackoverflow.com/questions/55434653/batch-normalization-doesnt-have-gradient-in-tensorflow-2-0
params = deeplabv3.trainable_weights + classifier.trainable_weights

updates = opt.get_updates(wce_loss, params)

train = tf.keras.backend.function(inputs=[X,Y], outputs=[wce_loss], updates=updates)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
deeplabv3.load_weights(fn_w_dlv3)
classifier.load_weights(fn_w_cls)

In [8]:
import matplotlib.pyplot as plt 
importlib.reload(data_utils)

#do_training = True

# Training on source domain
if do_training == True:
    try:
        deeplabv3.load_weights(fn_w_dlv3)
        classifier.load_weights(fn_w_cls)

        print("Successfully loaded model. Continuing training.")
    except:
        print("Could not load previous model weights. Is a new model present?")
        
    start_time = time.time()

    fig,ax = plt.subplots(1,figsize=(10,7))

    loss_history = []

    for itr in range(epochs):
    # for itr in range(1):
        source_train_data, source_train_labels = utils.sample_batch(source_data_dir, batch_size=batch_size, seed=itr)
        source_train_labels = tf.keras.utils.to_categorical(source_train_labels, num_classes=num_classes)

        loss_history.append(train(inputs=[source_train_data, source_train_labels ]))

        if np.isnan(np.sum(loss_history[-1])):
            print('ERROR. ENCOUNTERED NAN')
            break

        if itr%epoch_step == 0:
            if itr != 0:
                ax.clear()
                ax.plot(np.asarray(loss_history))

            ax.set_title("Training loss on source domain")
            ax.set_xlabel("Epoch")
            ax.set_ylabel("Loss")

            display.clear_output(wait=True)
            display.display(plt.gcf())
            time.sleep(1e-3)
            
        if itr % (epochs // 10) == 0 or itr == epochs - 1:
            deeplabv3.save_weights(fn_w_dlv3)
            classifier.save_weights(fn_w_cls)

    training_time = time.time() - start_time
else:
    deeplabv3.load_weights(fn_w_dlv3)
    classifier.load_weights(fn_w_cls)
    print("Loaded model weights")

Loaded model weights


In [9]:
import utils
importlib.reload(utils)

import data_utils_new

start_time = time.time()

source_cat_iou,source_miou = utils.compute_miou(source_data_dir, combined_, data_utils_new.label_ids_13, 0)

for k in source_cat_iou:
    print(k, source_cat_iou[k])
print(source_miou)

print('Computed ' + dataset + ' mIoU in', time.time() - start_time)

# road 0.8559070065772992
# sidewalk 0.9043721610536399
# building 0.8381948427759653
# wall 0.632677142340526
# fence 0.699526703820766
# pole 0.6073661307913504
# traffic light 0.7258756809296298
# traffic sign 0.6789546854786402
# vegetation 0.8526590005209183
# terrain 0.7779495588027171
# sky 0.9530645387149239
# person 0.7316948194592244
# rider 0.809945935261575
# car 0.5922120101790109
# truck 0.9211600020912953
# bus 0.9270433021188236
# train 0.8659149467023691
# motorcycle 0.6754852581619956
# bicycle 0.5876167577346231
# 0.7704010780797522
# Computed GTA5 mIoU in 1953.6462309360504

road 0.0
sidewalk 0.0
building 0.0
traffic light 0.0
traffic sign 0.0
vegetation 0.0
sky 0.0
person 0.0
rider 0.0
car 0.0
bus 0.0
motorcycle 0.0
bicycle 0.0
0.0
Computed SYNTHIA_VIPER_COMBINED_13_0331 mIoU in 3038.3256697654724


In [ ]:
importlib.reload(utils)

start_time = time.time()

cit_cat_iou,cit_miou = utils.compute_miou(cityscapes_data_dir, combined_, data_utils_new.label_ids_13, 0)

for k in cit_cat_iou:
    print(k, cit_cat_iou[k])
print(cit_miou)

print('Computed VIPER mIoU in', time.time() - start_time)

# road 0.4538541445555259
# sidewalk 0.17761519682584992
# building 0.5960010633542256
# wall 0.11064090337021892
# fence 0.08225737548235189
# pole 0.11701661235497808
# traffic light 0.2131517695419058
# traffic sign 0.085832471689785
# vegetation 0.7016830692115976
# terrain 0.05272756739697445
# sky 0.4267235195286716
# person 0.30570697364634025
# rider 0.03181916445284627
# car 0.4930313336164962
# truck 0.05910168795933546
# bus 0.052471835602942996
# train 0.003952731749887269
# motorcycle 0.07819570698033228
# bicycle 0.0073530220468457285
# 0.2131124289140585
# Computed CITYSCAPES mIoU in 249.57955932617188

In [ ]:
# import data_utils_new
# importlib.reload(utils)

# start_time = time.time()
# means, _, ct = utils.learn_gaussians(source_data_dir, deeplabv3, combined, batch_size, data_utils_new.label_ids_13)
# print("computed means in", time.time() - start_time)

# start_time = time.time()
# means, covs, ct = utils.learn_gaussians(source_data_dir, deeplabv3, combined, batch_size, data_utils_new.label_ids_13, \
#                                   initial_means=means)
# print("finished training gaussians in", time.time() - start_time)

# np.save("./extras/means_1024x512_" + backbone + "deeplabv3_" + dataset + ".npy", means)
# np.save("./extras/covs_1024x512_" + backbone + "deeplabv3_" + dataset + ".npy", covs)

In [ ]:
# for i in range(means.shape[0]):
#     print('=====================================================')
#     print(means[i])
#     print(covs[i])


In [ ]:
# means = np.load("./extras/means_1024x512_" + backbone + "deeplabv3_" + dataset + ".npy")
# covs = np.load("./extras/covs_1024x512_" + backbone + "deeplabv3_" + dataset + ".npy")

In [ ]:
# means = np.nan_to_num(means)
# means

In [ ]:
# Photo Version Replay Buffer Initialization
import replay_buffer_utils as rb_utils

start_time = time.time()
source_domain_portion = 0.1
replay_buffer = rb_utils.replay_buffer(source_data_dir, source_domain_portion)
print(time.time() - start_time)
print(replay_buffer.capacity)

In [12]:
# import umap.umap_ as umap
# import matplotlib.pyplot as plt
# # Generate data from the gmm model and plot it

# start_time = time.time()

# n_samples = np.ones(num_classes, dtype=int)
# n_samples[0] = 0
# n_samples *= 2000

# xx, yy = utils.sample_from_gaussians(means, covs, n_samples=n_samples)

# NUM_COLORS = num_classes

# reducer = umap.UMAP()

# umap_embedding = reducer.fit_transform(xx)

# plt.figure(figsize=(16,14))
# cm = plt.get_cmap('gist_rainbow')

# shift = 1 / len(data_utils_new.label_ids_13.keys())
# idx = 0
# for label in data_utils_new.label_ids_13:
#     ind = yy == data_utils_new.label_ids_13[label]
    
#     plt.scatter(umap_embedding[:,0][ind], umap_embedding[:,1][ind], label=label, \
#                 color=cm(1.*idx/NUM_COLORS))
#     idx += 1

# plt.title("Embedding scatter-plot")
# plt.legend()
    
# plt.show()

# print(time.time() - start_time)

In [ ]:
importlib.reload(wasserstein_utils)

#Z_s = tf.keras.layers.Input(shape=(img_shape[0], img_shape[1], num_classes,) )
Z_s = tf.keras.layers.Input(shape=(img_shape[0], img_shape[1], 3,))
Y_s = tf.keras.backend.placeholder(shape=(None, img_shape[0], img_shape[1], num_classes), dtype='float32') #labels of input images oneHot
lambda2 = 5e-1

loss_function = losses.weighted_ce_loss(num_classes, 0)
#wce_loss = loss_function(Y_s, classifier(Z_s), from_logits=False)
wce_loss = loss_function(Y_s, classifier(deeplabv3(Z_s)), from_logits=False)


# Wasserstein matcing loss
theta = tf.keras.backend.placeholder(shape = (num_projections, num_classes), dtype='float32')
#matching_loss = wasserstein_utils.sWasserstein_hd(deeplabv3(X), Z_s, theta, nclass=num_classes, Cp=None, Cq=None,)
matching_loss = wasserstein_utils.sWasserstein_hd(deeplabv3(X), deeplabv3(Z_s), theta, nclass=num_classes, Cp=None, Cq=None,)

# Overall loss is a weighted combination of the two losses
total_loss = wce_loss + lambda2*matching_loss

params = deeplabv3.trainable_weights + classifier.trainable_weights

# Optimizer and training setup
opt = tf.keras.optimizers.Adam(lr=1e-4, epsilon=1e-6, decay=1e-6)

updates = opt.get_updates(total_loss, params)
train = tf.keras.backend.function(inputs=[X,Z_s,Y_s,theta], outputs=[total_loss, wce_loss, matching_loss], updates=updates)

In [ ]:
loss = []
target_miou = []

deeplabv3.load_weights(fn_w_dlv3)
classifier.load_weights(fn_w_cls)

In [ ]:
fig,ax=plt.subplots(2,figsize=(15,10))

batch_size = 1

thres = .99
#epochs2 = 50000
epochs2 = 2000
epochstep = 100

for itr in range(epochs2):
    
    if itr%100 == 0:
        print('Iteration: {}'.format(itr))
    
    target_train_data, target_train_labels = utils.sample_batch(cityscapes_data_dir, batch_size=batch_size, seed=itr)

    # make sure the #samples from gaussians match the distribution of the labels
    n_samples = np.zeros(num_classes, dtype=int)
    cls, ns = np.unique(target_train_labels, return_counts=True)
    for i in range(len(cls)):
        if cls[i] == 0:
            continue
        n_samples[cls[i]] = ns[i]

    if np.sum(n_samples) % np.prod(img_shape) != 0:
        remaining = np.prod(img_shape[:-1]) - np.sum(n_samples) % np.prod(img_shape[:-1])

        aux = np.copy(n_samples) / np.sum(n_samples)
        aux *= remaining
        aux = np.floor(aux).astype('int')
        
        n_samples += aux

        # in case there are extra samples left, dump them on the highest represented class
        n_samples[np.argmax(n_samples)] += remaining - np.sum(aux)

###   replay buffer statistics version
#     Yembed,Yembedlabels = utils.sample_from_gaussians(means, covs, n_samples = n_samples)
#     Yembed = Yembed.reshape(-1, img_shape[0], img_shape[1], num_classes)
#     Yembedlabels = Yembedlabels.reshape(-1, img_shape[0], img_shape[1])

#     Yembedlabels = tf.keras.utils.to_categorical(Yembedlabels, num_classes=num_classes)

#     theta_instance = tf.keras.backend.variable(wasserstein_utils.generateTheta(num_projections,num_classes))
#     loss.append(train(inputs=[target_train_data, Yembed, Yembedlabels, theta_instance]))
###   statistics version ends here


###   replay buffer photo version
    X, Y = rb_utils.random_sample_replay(replay_buffer, 1)
    Y = tf.keras.utils.to_categorical(Y, num_classes=num_classes)
    theta_instance = tf.keras.backend.variable(wasserstein_utils.generateTheta(num_projections,num_classes))
    # print(X.shape)
    # print(Y.shape)
    loss.append(train(inputs=[target_train_data, X, Y, theta_instance]))
###   replay buffer photo version ends here

    target_miou.append(combined_.evaluate(target_train_data, target_train_labels, verbose=False)[-1] * 20/19)
    
    if itr%epochstep==0:
        # Debug info. First, the mIoU. Second, the categorical CE loss (ignoring class weights and containing) 
        # the ignore class
        if itr != 0:
            ax[0].clear()
            
            ll = np.asarray(loss)
            ax[0].plot(ll[:,0], label='total loss')
            ax[0].plot(ll[:,1], label='ce loss')
            ax[0].plot(ll[:,2] * lambda2, label='wasserstein loss')
            ax[0].legend()
            
        ax[0].set_title("Loss")
        ax[0].set_xlabel("Epochs")
        ax[0].set_ylabel("Loss")
        
        if itr != 0:
            ax[1].clear()
            ax[1].plot(np.asarray(target_miou))
        
        ax[1].set_title("MIOU on target domain")
        ax[1].set_xlabel("Epochs")
        ax[1].set_ylabel("Mean IOU")
        
        display.display(plt.gcf())
        display.clear_output(wait=True)
        
        time.sleep(1e-3)

In [ ]:
import utils
importlib.reload(utils)

start_time = time.time()

source_cat_iou,source_miou = utils.compute_miou(source_data_dir, combined_, data_utils.label_ids_13, 0)

for k in source_cat_iou:
    print(k, source_cat_iou[k])
print(source_miou)

print('Computed ' + dataset + ' mIoU in', time.time() - start_time)

# road 0.6675523660118785
# sidewalk 0.28336359271439726
# building 0.6076388775470897
# wall 0.2672064026700072
# fence 0.17232300484931942
# pole 0.2735294951039704
# traffic light 0.32147051090247314
# traffic sign 0.07400171986539407
# vegetation 0.5965591779805058
# terrain 0.2889806932148324
# sky 0.8603173835723591
# person 0.3489449704700763
# rider 0.17189785226832072
# car 0.5761907340114261
# truck 0.2955718142857747
# bus 0.14318027819603177
# train 0.033001835642511974
# motorcycle 0.12385339561083183
# bicycle 0.025308995240981697
# 0.3226785842188517
# Computed GTA5 mIoU in 1957.6215891838074

In [ ]:
start_time = time.time()

cit_cat_iou,cit_miou = utils.compute_miou(cityscapes_data_dir, combined_, data_utils.label_ids_13, 0)

for k in cit_cat_iou:
    print(k, cit_cat_iou[k])
print(cit_miou)

print('Computed VIPER mIoU in', time.time() - start_time)

# road 0.7883116582422506
# sidewalk 0.6215464735885322
# building 0.751691007583865
# wall 0.3993120030432491
# fence 0.39173486089325144
# pole 0.302509334397975
# traffic light 0.3019573318279201
# traffic sign 0.38603017233885695
# vegetation 0.7792104073113252
# terrain 0.484321694588526
# sky 0.8323752669785053
# person 0.5206867614087272
# rider 0.2377222662760571
# car 0.8020324193022697
# truck 0.4754062370048565
# bus 0.593789975462107
# train 0.5966522990567177
# motorcycle 0.3119814031424603
# bicycle 0.47793033593638945
# 0.5292211530728338
# Computed CITYSCAPES mIoU in 232.43373942375183

In [ ]:
deeplabv3.save_weights(fn_w_adapted_dlv3)
classifier.save_weights(fn_w_adapted_cls)

In [ ]:
deeplabv3.load_weights(fn_w_adapted_dlv3)
classifier.load_weights(fn_w_adapted_cls)